In [2]:
# 최종 medication.csv 코드
import pandas as pd
import re
from langchain.schema import Document

def parse_medication_description(text):
    """
    DESCRIPTION 문자열을 파싱하여
      code            : NDA/ANDA 코드
      drug_names      : [성분명1, 성분명2, …]
      dose_amounts    : [용량1, 용량2, …]
      form            : 제형 (예: 'Oral Tablet', 'Topical Cream', ...)
      route_or_extra  : 제형 뒤 추가정보
    """
    s = str(text).strip()
    res = {"code":"", "drug_names":[], "dose_amounts":[], "form":"", "route_or_extra":""}

    # 1) 제품 코드
    m = re.search(r'\b(NDA|ANDA)\d+\b', s)
    if m:
        res["code"] = m.group()

    # 2) 제형(form)과 뒤 추가정보 분리 — 수정된 부분
    form_pattern = re.compile(
        r'\b(?:'
        r'Extended Release Oral Tablet|Extended Release Tablet|Oral Tablet|Sublingual Tablet|Chewable Tablet|Oral Capsule|'
        r'Intravenous Injection|Intramuscular Injection|Inhalation Solution|Mucosal Spray|Drug Implant|'
        r'Transdermal System|Injectable Suspension|Topical Cream|Intrauterine System|Ophthalmic Solution|'
        r'Tablet|Capsule|Injection|Solution|Cream|Pack|Inhaler|Syringe|Suspension|Auto-Injector|Oral Suspension|'
        r'Patch|System|Implant|Spray|Drop |Oral Gel|Injectable Solution|Pen Injector|Vaginal Ring|Vaginal System'
        r')\b',
        flags=re.IGNORECASE
    )
    fm = form_pattern.search(s)
    if fm:
        # 2-a) form 에 전체 매칭 문자열(예: "Oral Tablet")을 title()-통일해서 저장
        res["form"] = fm.group().title()
        # 2-b) form 뒤 나오는 모든 텍스트를 추가 정보로
        tail = s[fm.end():].strip(" -/[]")
        res["route_or_extra"] = tail
        # 2-c) 뒤를 잘라내고, 앞부분(성분+용량)만 남겨서 다음 단계 처리
        s = s[:fm.end()]

    # 3) “ / ” (공백-슬래시-공백) 로만 세그먼트 분리
    split_pattern = re.compile(
        r'''
        \s+/\s+                 # 공백 / 공백
        (?!(?:                   # negative lookahead 시작
            MG(?:/(?:ML|MG|ACTUAT|HR))?   # MG, MG/ML, MG/MG, MG/ACTUAT, MG/HR
          | ML(?:/ML)?           # ML, ML/ML
          | UNT/ML               # UNT/ML
          | ACTUAT               # ACTUAT
        )\b)                     # 단어 경계
        ''',
        flags=re.IGNORECASE | re.VERBOSE
    )
    segments = split_pattern.split(s)


    # 4) 각 세그먼트에서 성분명 & 용량 추출
    dose_pat = (
    # 1) 숫자/숫자 + MG (예: 0.25/0.05 MG/ACTUAT)
    r'\d+(?:\.\d+)?/\d+(?:\.\d+)?\s*MG(?:/ACTUAT)?'
    r'|'
    # 2) 숫자 + MG/ACTUAT (예: 0.25 MG/ACTUAT)
    r'\d+(?:\.\d+)?\s*MG(?:/ACTUAT)'
    r'|'
    # 3) 숫자 + ACTUAT (예: 60 ACTUAT)
    r'\d+\s*ACTUAT'
    r'|'
    # 4) 그 외 숫자 + 단위 (MG, ML, HR, Day 등)
    r'\d+(?:\.\d+)?\s*'
        r'(?:'
            r'MG(?:/(?:ML|MG|HR))?'    # MG, MG/ML, MG/MG, MG/HR
          r'|ML(?:/ML)?'               # ML, ML/ML
          r'|UNT/ML'                   # UNT/ML
          r'|%|HR|MG/HR'               # %, HR, MG/HR
          r'|Day(?:s)?'                # Day, Days
        r')'
    )

    for seg in segments:
        seg = seg.strip()
        # a) 용량
        doses = re.findall(dose_pat, seg, flags=re.IGNORECASE)
        res["dose_amounts"].append(" / ".join(doses) if doses else "")
        # b) 성분명: 용량, 코드명, form 토큰 제거
        name = seg
        for d in doses:
            name = re.sub(re.escape(d), "", name, flags=re.IGNORECASE)
        name = re.sub(r'\b(NDA|ANDA)\d+\b', "", name, flags=re.IGNORECASE)
        if res["form"]:
            name = re.sub(re.escape(fm.group()), "", name, flags=re.IGNORECASE)
        res["drug_names"].append(name.strip(" -"))

    return res

    # --- CSV 읽고 RAG doc 생성까지 통합 예시 ---
df = pd.read_csv("medications.csv")
df["START"] = pd.to_datetime(df["START"])
df["STOP"]  = pd.to_datetime(df["STOP"], errors="coerce")

medication_docs = []
for pid, group in df.groupby("PATIENT"):
    lines = []
    for desc, sub in group.groupby("DESCRIPTION"):
        info     = parse_medication_description(desc)
        drugs    = info["drug_names"]      # ex. ["Amoxicillin","Clavulanate"]
        doses    = info["dose_amounts"]    # ex. ["250 MG","125 MG"]
        form     = info["form"]
        reason   = sub["REASONDESCRIPTION"].dropna().unique()
        reason_t = f"(사유: {', '.join(reason)})" if len(reason) else ""

        # 복용 기간 & 횟수
        st = sub["START"].min().strftime("%Y-%m")
        en = sub["STOP"].max()
        if pd.isna(en):
            period = f"{st} ~ "
        else:
            en = en.strftime("%Y-%m")
            period = st if st==en else f"{st} ~ {en}"
        cnt = sub["DISPENSES"].sum()

        # 한 줄 문장
        lines.append(
            f"- 성분: {', '.join(drugs)}  "
            f"용량: {', '.join(doses)}  "
            f"제형: {form}  "
            f"기간: {period}  "
            f"횟수: {cnt}회  "
            f"{reason_t}"
        )

    combined = f"환자 ID: {pid[:8]}\n환자는 다음 약물을 복용했습니다:\n" + "\n".join(lines)
    medication_docs.append(Document(
        page_content=combined,
        metadata={"patient_id":pid[:8],"table":"medications"}
    ))

print("환자 수:", len(medication_docs))
print(medication_docs[0].page_content)

환자 수: 1158
환자 ID: 0063c9a6
환자는 다음 약물을 복용했습니다:
- 성분: heparin sodium  porcine  용량: 1 ML / 5000 UNT/ML  제형: Injection  기간: 2008-05  횟수: 1회  
- 성분: Furosemide  용량: 10 ML / 10 MG/ML  제형: Injection  기간: 2010-04  횟수: 10회  (사유: Chronic congestive heart failure (disorder))
- 성분: Propofol  용량: 100 ML / 10 MG/ML  제형: Injection  기간: 2008-05  횟수: 1회  
- 성분: Ondansetron  용량: 2 ML / 2 MG/ML  제형: Injection  기간: 2008-05  횟수: 1회  
- 성분: metoprolol succinate  용량: 24 HR / 100 MG  제형: Extended Release Oral Tablet  기간: 2008-05 ~   횟수: 24회  
- 성분: protamine sulfate (USP)  용량: 25 ML / 10 MG/ML  제형: Injection  기간: 2008-05  횟수: 1회  
- 성분: SUFentanil  용량: 5 ML / 0.05 MG/ML  제형: Injection  기간: 2008-05  횟수: 1회  
- 성분: Clopidogrel  용량: 75 MG  제형: Oral Tablet  기간: 2008-05 ~   횟수: 24회  
- 성분: Furosemide  용량: 40 MG  제형: Oral Tablet  기간: 2009-10 ~   횟수: 6회  (사유: Chronic congestive heart failure (disorder))
- 성분: Ibuprofen  용량: 200 MG  제형: Oral Tablet  기간: 2008-11 ~ 2008-12  횟수: 1회  
- 성분: Midazolam  용량: 1 MG/ML  제형: In

In [3]:
import numpy as np

# 각 문서의 문자열 길이 측정
lengths = [len(doc.page_content) for doc in medication_docs]

# 통계 계산
avg_len = np.mean(lengths)
median_len = np.median(lengths)
min_len = np.min(lengths)
max_len = np.max(lengths)

# 출력
print("총 문서 수:", len(lengths))
print(f"평균 문자열 길이: {avg_len:.2f}")
print(f"중앙값 문자열 길이: {median_len}")
print(f"최소 문자열 길이: {min_len}")
print(f"최대 문자열 길이: {max_len}")

총 문서 수: 1158
평균 문자열 길이: 785.55
중앙값 문자열 길이: 654.5
최소 문자열 길이: 102
최대 문자열 길이: 3052
